In [2]:
!pip install pandas numpy sqlalchemy psycopg2-binary scikit-learn lightgbm joblib


   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 1.5/1.5 MB 9.5 MB/s eta 0:00:00


In [21]:
import os
import pandas as pd
import numpy as np
import joblib
from datetime import timedelta

#DB
import psycopg2
from sqlalchemy import create_engine

# ML
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error, mean_squared_error
import lightgbm as lgb

DB_HOST = os.getenv("DB_HOST", "proyectohabreee.cv4ea0syasip.eu-north-1.rds.amazonaws.com")
DB_PORT = os.getenv("DB_PORT", "5432")
DB_NAME = os.getenv("DB_NAME", "postgres")
DB_USER = os.getenv("DB_USER", "postgres")
DB_PASS = os.getenv("DB_PASS", "proyectohabree123")
TABLE_NAME = "demanda_ree"

# Paths
MODEL_PATH = "lgb_demand_model.joblib"
FEATURES_PATH = "features_cols.txt"

# ---------------------------
# 1) EXTRAER DATOS
# ---------------------------
def load_data_from_rds(limit_rows=None, date_from=None, date_to=None):
    """
    Conecta a la BBDD y devuelve un DataFrame con columnas: id, fecha, demanda_mw
    Opcional: limit_rows para desarrollo local.
    """
    conn_str = f"postgresql+psycopg2://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
    engine = create_engine(conn_str, connect_args={"connect_timeout": 10})
    query = f"SELECT id, fecha, demanda_mw FROM {TABLE_NAME} ORDER BY fecha ASC" 
    if limit_rows:
        query += f" LIMIT {limit_rows}"
    df = pd.read_sql(query, engine)
    return df


In [22]:

# ---------------------------
# 2) PREPROCESS / FEATURE ENGINEERING
# ---------------------------
def preprocess_and_features(df):
    """
    - Convierte fecha a datetime y ordena.
    - Crea features temporales y lags/rolling.
    - Devuelve DataFrame listo para modelado.
    """
    df = df.copy()
    df['fecha'] = pd.to_datetime(df['fecha']).dt.date
    df['fecha'] = pd.to_datetime(df['fecha'])
    df = df.sort_values('fecha').drop_duplicates(subset='fecha').reset_index(drop=True)
          
    # Features temporales
    df['dayofweek'] = df['fecha'].dt.dayofweek
    df['month'] = df['fecha'].dt.month
    df['is_weekend'] = df['dayofweek'].isin([5,6]).astype(int)
    
    # Crea lags: 1, 7, 14, 30 (ayer, una semana antes, dos semanas antes, un mes antes)
    lags = [1, 7, 14, 30]  # personalizable
    for l in lags:
        df[f'lag_{l}'] = df['demanda_mw'].shift(l)
    
    # rolling means
    windows = [7, 14, 30]
    for w in windows:
        df[f'roll_mean_{w}'] = df['demanda_mw'].shift(1).rolling(window=w, min_periods=1).mean()
        df[f'roll_std_{w}'] = df['demanda_mw'].shift(1).rolling(window=w, min_periods=1).std().fillna(0)
    
    # Drop initial rows with NaN from lags
    df = df.dropna().reset_index(drop=True)
    return df



In [34]:

# ---------------------------
# 3) TRAIN / VALIDATION SPLIT (temporal)
# ---------------------------
def temporal_train_test_split(df, val_size_days=500, test_size_days=500):
    """
    Divide en train/val/test en orden temporal.
    val_size_days, test_size_days: cuánto reservar para validación y test (en días).
    """
    n = len(df)
    if n < (val_size_days + test_size_days + 10):
        raise ValueError("Muy pocos datos para dividir en train/val/test.")

    train_end = n - val_size_days - test_size_days
    val_end = n - test_size_days

    train = df.iloc[:train_end].copy()
    val = df.iloc[train_end:val_end].copy()
    test = df.iloc[val_end:].copy()
    
    return train, val, test


In [35]:

# ---------------------------
# 4) MODEL TRAIN
# ---------------------------
def train_lightgbm(train_df, val_df, features, target='demanda_mw', params=None):
    train_data = lgb.Dataset(train_df[features], label=train_df[target])
    val_data = lgb.Dataset(val_df[features], label=val_df[target])
    default_params = {
        "objective": "regression",
        "metric": "rmse",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "learning_rate": 0.005,
        "num_leaves": 31,
        "n_estimators": 1000,
    }
    if params:
        default_params.update(params)
    gbm = lgb.train(default_params, 
                    train_data, 
                    valid_sets=[val_data],
                    callbacks=[
                        lgb.early_stopping(stopping_rounds=50),
                        lgb.log_evaluation(period=50)  # imprime cada 50 iteraciones
                    ] 
    )
    return gbm


In [39]:

# ---------------------------
# 5) EVALUATION
# ---------------------------
def evaluate_model(model, df, features, target='demanda_mw'):
    preds = model.predict(df[features])
    mae = mean_absolute_error(df[target], preds)
    rmse = mean_squared_error(df[target], preds, squared=False)
    mape = np.mean(np.abs((df[target] - preds) / (df[target] + 1e-9))) * 100
    accuracy = 100 - mape
    return {"MAE: ": mae, "RMSE: ": rmse, "MAPE(%): ": mape, "ACC (%): ": accuracy}, preds


In [42]:

# ---------------------------
# 6) PREDICT FUTURE (horizon steps)
# ---------------------------
def predict_future(model, last_df, features, horizon=7):
    """
    last_df: DataFrame with the latest rows including features (must include fecha and demanda_mw).
    Devuelve DataFrame con predicciones para next `horizon` steps (assume hourly).
    Procedimiento: iterativo, se genera features usando predicción anterior para lags.
    """
    last_df = last_df.copy().sort_values('fecha').reset_index(drop=True)
    preds = []
    df_work = last_df.copy()

    for step in range(horizon):
        next_time = df_work['fecha'].iloc[-1] + pd.Timedelta(days=1)
        row = {"fecha": next_time}
        row['dayofweek'] = next_time.dayofweek
        row['month'] = next_time.month
        row['is_weekend'] = int(next_time.dayofweek in [5,6])
        
        # lags: tomar de df_work
        for l in [1,7,14,30]:
            if l <= len(df_work):
                row[f'lag_{l}'] = df_work['demanda_mw'].iloc[-l]
            else:
                row[f'lag_{l}'] = np.nan

        # rolling means/std using last values
        for w in [7,14,30]:
            vals = df_work['demanda_mw'].shift(1).iloc[-w:] if len(df_work) >= 1 else []
            if len(vals) > 0:
                row[f'roll_mean_{w}'] = float(vals.mean())
                row[f'roll_std_{w}'] = float(vals.std()) if len(vals) > 1 else 0.0
            else:
                row[f'roll_mean_{w}'] = np.nan
                row[f'roll_std_{w}'] = np.nan
        
        row_df = pd.DataFrame([row])
        # fill NaNs with reasonable values (e.g., mean of each column from df_work)
        for c in features:
            if c not in row_df.columns:
                row_df[c] = np.nan
        
        # align columns
        row_df = row_df[features].fillna(df_work[features].mean())
        pred = model.predict(row_df)[0]
        
        # append
        new_row = {"fecha": next_time, "demanda_mw": pred}
        preds.append(new_row)
        # add to df_work for next iteration
        df_work = pd.concat([df_work, pd.DataFrame([new_row])], ignore_index=True)

    return pd.DataFrame(preds)




In [43]:

# ---------------------------
# MAIN FLOW
# ---------------------------
def main():
    # 1. Cargar
    df = load_data_from_rds()
    print("Registros extraídos:", len(df))
    
    # 2. Preprocess
    df_proc = preprocess_and_features(df)
    print("Registros tras preprocess:", len(df_proc))
    
    # 3. Split
    train, val, test = temporal_train_test_split(df_proc)
    print("Tamaños -> train:", len(train), "val:", len(val), "test:", len(test))
    
    # 4. Features list
    # Excluir columnas id, fecha, demanda_mw
    excluded = ['id', 'fecha', 'demanda_mw']
    features = [c for c in df_proc.columns if c not in excluded]

    print("Columnas disponibles:", df_proc.columns.tolist())
    print("Features seleccionadas:", features)
    print("Tamaño TRAIN:", train.shape)
    print("Tamaño VAL:", val.shape)
    
    # Guardar lista de features
    with open(FEATURES_PATH, 'w') as f:
        f.write("\n".join(features))
    
    # 5. Entrenar LightGBM
    model = train_lightgbm(train, val, features)
    
    # guardar modelo
    joblib.dump(model, MODEL_PATH)
    print("Modelo guardado en:", MODEL_PATH)
    
    # 6. Evaluar en train/val/test
    print("Evaluación TRAIN: ", evaluate_model(model, train, features)[0])
    print("Evaluación VAL: ", evaluate_model(model, val, features)[0])
    print("Evaluación TEST: ", evaluate_model(model, test, features)[0])
   
    # 7. Ejemplo: predecir 7 dias futuros usando los últimos datos del test set
    last_block = pd.concat([train, val, test]).iloc[-180:].copy()  # usar ventana de referencia
    preds_future = predict_future(model, last_block, features, horizon=7)
    print("Predicciones próximos 7 dias:")
    print(preds_future.head(7))

if __name__ == "__main__":
    main()



Registros extraídos: 5384
Registros tras preprocess: 5354
Tamaños -> train: 4354 val: 500 test: 500
Columnas disponibles: ['id', 'fecha', 'demanda_mw', 'dayofweek', 'month', 'is_weekend', 'lag_1', 'lag_7', 'lag_14', 'lag_30', 'roll_mean_7', 'roll_std_7', 'roll_mean_14', 'roll_std_14', 'roll_mean_30', 'roll_std_30']
Features seleccionadas: ['dayofweek', 'month', 'is_weekend', 'lag_1', 'lag_7', 'lag_14', 'lag_30', 'roll_mean_7', 'roll_std_7', 'roll_mean_14', 'roll_std_14', 'roll_mean_30', 'roll_std_30']
Tamaño TRAIN: (4354, 16)
Tamaño VAL: (500, 16)
Training until validation scores don't improve for 50 rounds
[50]	valid_0's rmse: 64201.4
[100]	valid_0's rmse: 53571.8
[150]	valid_0's rmse: 45851.6
[200]	valid_0's rmse: 40100.5
[250]	valid_0's rmse: 35775.4
[300]	valid_0's rmse: 32755.8
[350]	valid_0's rmse: 30308
[400]	valid_0's rmse: 28683.3
[450]	valid_0's rmse: 27618.5
[500]	valid_0's rmse: 26797.8
[550]	valid_0's rmse: 26182.2
[600]	valid_0's rmse: 25741.6
[650]	valid_0's rmse: 25421.

d:\ANACONDA_HACKABOSS\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
d:\ANACONDA_HACKABOSS\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
d:\ANACONDA_HACKABOSS\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
